## MTG Stapler
Jupyter Notebook to test and download all the COMPETITIVE cards.

Future TO DO:
* Add support for MTG Goldfish Lists
* Add support for Archidekt
* Add Brewer's Corner
* Get only cards in decklists
* Add Maybeboard and Sideaboard Classification
* Make more specific scrapping to omit sample hand

In [8]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException
import pandas as pd
import re
import os
from tqdm.notebook import tqdm


## Scrapping cEDH lists from the decklist database

In [3]:
cedh_database =  'https://cedh-decklist-database.com'
client = uReq(cedh_database) # opening connection 
page_html = client.read() # content to variable
client.close() # closes connection 
page_soup = soup(page_html, 'html.parser') # html parsing 
containers = page_soup.findAll("div",{"class": "ddb-section"})
htmls = []
for c in containers:
    if c.get_text().strip() == 'COMPETITIVE':
        x = c.parent.parent.find("ul", {"class": "ddb-decklists"})
        lis = x.findAll('li')
        for li in lis:
            htmls.append(li.a["href"])

Checking not moxfield lists

In [15]:
[i for i in htmls if 'moxfield' not in i]

['https://www.mtggoldfish.com/deck/3982753#paper',
 'https://www.mtggoldfish.com/deck/3982746#paper',
 '',
 '',
 '',
 'https://archidekt.com/decks/74512#A_Song_of_Turns_and_Dongers']

## Cleaning URLS
* Remove trailing / 
* Remove trailing /primer

In [6]:
def second_group(m):
    return m.group(1)
htmls = list(map(lambda x: re.sub(r'(.*)/$|(.*)/primer$',second_group,x ), htmls))

In [37]:
htmls

['https://www.moxfield.com/decks/1vqfLIEtWkOnZo-ywKQtKA',
 'https://www.moxfield.com/decks/HmRJ5ju54USIf9UVsl5kwQ',
 'https://www.moxfield.com/decks/belqftDqDEWVEZUMf88PCw',
 'https://www.moxfield.com/decks/yRsS18tYsE-jVgqmK7_Z0w',
 'https://www.moxfield.com/decks/K6JZovGucE-W8rAnoVgmNA',
 'https://www.moxfield.com/decks/SsKY4t8Mnk66xHA3cOokuQ',
 'https://www.moxfield.com/decks/uxtkikNzW0mTluQuOGF4eg',
 'https://www.moxfield.com/decks/rdb6vhTBQk2dAYDAdnt_jw',
 'https://www.moxfield.com/decks/jNFzjXtnu0ac1AoBz80Vbw',
 'https://www.moxfield.com/decks/4iZV3rLTGkmX2H_FFP_1dQ',
 'https://www.moxfield.com/decks/EwS7lyEQN0mLtomfKlwoMQ',
 'https://www.moxfield.com/decks/CuKho0dh4UaVpYJWy3xG4A',
 'https://www.moxfield.com/decks/5_b4DltiAkaMGDI3rv3TxA',
 'https://www.moxfield.com/decks/VtL56p-yU0-bRGtaJFVqHw',
 'https://www.moxfield.com/decks/N1VNylbgskmYWa09r26ygQ',
 'https://www.moxfield.com/decks/Y2cdFYZw_0-isKafmKXgNw',
 'https://www.moxfield.com/decks/hbLMs7xcfUyun2nxK-sRlw',
 'https://www.

## SCRAPING DECKLISTS

Only Moxfield COMPETITIVE lists

In [9]:
cards = []
chromedriver_path=os.path.join(os.getcwd(), "chromedriver.exe")
options = Options()
options.headless = True
options.add_experimental_option('excludeSwitches', ['enable-logging']) #remove logging message
driver = webdriver.Chrome(chromedriver_path, options= options)
for decklist in tqdm(htmls):
    if 'moxfield' in decklist: 
        driver.get(decklist)
        try:
            myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'viewMode')))
        except TimeoutException:
            print ("Loading took too much time!",decklist)
            continue
        view_mode = driver.find_element_by_id('viewMode')
        mode = view_mode.get_attribute('value')
        page_source = driver.page_source
        page_soup = soup(page_source, 'html.parser')
        title = page_soup.find("span", {"class": "deckheader-name"}).get_text()
        deck_info = {'Link': decklist, 'Title':title }
        if mode == 'table':
            containers = page_soup.findAll("tr",{"class": "table-deck-row"})
            deck_cards = []
            #TODO: Find only decklist cards
            for c in containers:
                c_name = {'Card Name': c.a.get_text()}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)
        elif mode == 'visual':
            containers = page_soup.findAll("div",{"class": "decklist-card-phantomsearch"})
            deck_cards = []
            #TODO: Find only decklist cards
            for c in containers:
                c_name = {'Card Name': c.get_text()}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)
        elif mode == 'stacks':
            containers = page_soup.findAll("div",{"class": "img-card-stack"})
            deck_cards = []
            #TODO: Find only decklist cards
            for c in containers:
                c_name = {'Card Name': c.img['alt']}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)
driver.close()

  0%|          | 0/211 [00:00<?, ?it/s]

Loading took too much time! https://www.moxfield.com/decks/gMN5-x51uE2eSlQbKWbz-A
Loading took too much time! https://www.moxfield.com/decks/4fbiNVFpr0edHL5Y3IKecg
Loading took too much time! https://www.moxfield.com/decks/Sh9PHfRR202_DRLRxxxhEQ
Loading took too much time! https://www.moxfield.com/decks/BPErFu24x0S37bFE_Ibp8w


In [13]:
df_cards = pd.DataFrame(cards)
df_cards['c_count'] = df_cards['Cards'].apply(len)
df_cards.sort_values(by='c_count', ascending=True)

,Link,Title,Cards,c_count
183,https://www.moxfield.com/decks/G0YVGWD68keygYq...,Jace High Tide,"[{'Card Name': 'Jace, Vryn's Prodigy'}, {'Card...",84
34,https://www.moxfield.com/decks/rohOeIs-dEmA1LA...,Heliod II: Sunshine Boogaloo,"[{'Card Name': 'Heliod, Sun-Crowned'}, {'Card ...",85
192,https://www.moxfield.com/decks/ilvW7g3hsUio09K...,Wanderer's Song,"[{'Card Name': 'Yisan, the Wanderer Bard'}, {'...",87
164,https://www.moxfield.com/decks/7Og0NOMESUaFwAs...,Mommy Marwyn (cEDH Elves),"[{'Card Name': 'Marwyn, the Nurturer'}, {'Card...",88
202,https://www.moxfield.com/decks/PQPz_KSrV0S5_Hu...,Selvala Brostorm,"[{'Card Name': 'Selvala, Heart of the Wilds'},...",88
...,...,...,...,...
197,https://www.moxfield.com/decks/yBih2LlY3UmOJkv...,Varolz Hulk,"[{'Card Name': 'Varolz, the Scar-Striped'}, {'...",123
53,https://www.moxfield.com/decks/C5xyxwytW0eaKZk...,Turbo Seeker Inalla,"[{'Card Name': 'Inalla, Archmage Ritualist'}, ...",124
52,https://www.moxfield.com/decks/yrFKOscy6kWLOZ6...,Storm Seeker [cEDH],"[{'Card Name': 'Inalla, Archmage Ritualist'}, ...",125
67,https://www.moxfield.com/decks/DRp3K4w94Uu-8JY...,The Pet Cemetery,"[{'Card Name': 'Tana, the Bloodsower'}, {'Card...",125


## Normalizing JSON
Adding support

In [29]:
df = pd.json_normalize(cards,
    record_path='Cards', meta=['Link', 'Title'])
df = df.drop_duplicates()
df

,Card Name,Link,Title
0,Sidar Kondo of Jamuraa,https://www.moxfield.com/decks/1vqfLIEtWkOnZo-...,Tymna/Sidar Raza
1,Tymna the Weaver,https://www.moxfield.com/decks/1vqfLIEtWkOnZo-...,Tymna/Sidar Raza
2,Arbor Elf,https://www.moxfield.com/decks/1vqfLIEtWkOnZo-...,Tymna/Sidar Raza
3,Avacyn's Pilgrim,https://www.moxfield.com/decks/1vqfLIEtWkOnZo-...,Tymna/Sidar Raza
4,Birds of Paradise,https://www.moxfield.com/decks/1vqfLIEtWkOnZo-...,Tymna/Sidar Raza
...,...,...,...
21758,Underground Sea,https://www.moxfield.com/decks/tLPpgjrZ7EWlJZ4...,Opus Thief
21759,Verdant Catacombs,https://www.moxfield.com/decks/tLPpgjrZ7EWlJZ4...,Opus Thief
21760,Volcanic Island,https://www.moxfield.com/decks/tLPpgjrZ7EWlJZ4...,Opus Thief
21761,Watery Grave,https://www.moxfield.com/decks/tLPpgjrZ7EWlJZ4...,Opus Thief


In [30]:
#convert dataframe of decklists with cards to a dataframe of cards with decklists
df_1 = df.groupby('Card Name')['Title'].apply(list).reset_index()
df_2 = df.groupby('Card Name')['Link'].apply(list).reset_index()
df_12 = pd.merge(df_1, df_2, on='Card Name', how='inner')
df_12.head()

,Card Name,Title,Link
0,Abandon Reason,[Filter Anje cEDH Primer],[https://www.moxfield.com/decks/PFGBjM5BnESfH9...
1,Abeyance,[Heliod II: Sunshine Boogaloo],[https://www.moxfield.com/decks/rohOeIs-dEmA1L...
2,Abrade,"[Frozen Scepter Parun 2.0, Rielle Adaptative W...",[https://www.moxfield.com/decks/belqftDqDEWVEZ...
3,Abrupt Decay,"[Tymna/Sidar Raza, The King's Chain, Medium Mi...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...
4,Abundance,[Simic Ad Naus in the CZ],[https://www.moxfield.com/decks/Vva2blXPDEWTPC...


In [31]:
df_12['Card Name'].nunique()

1536

In [34]:
card_vc = df['Card Name'].value_counts()
card_vc = card_vc.reset_index()
card_vc.rename( columns={'index' :'Card Name', 'Card Name':'Occurrences'}, inplace=True )#columns names
card_vc

,Card Name,Occurrences
0,Mana Crypt,201
1,Chrome Mox,200
2,Sol Ring,199
3,Mox Diamond,198
4,Polluted Delta,176
...,...,...
1531,Dizzy Spell,1
1532,Crippling Fear,1
1533,Wrath of God,1
1534,Blade Historian,1


In [35]:
full_cards = pd.merge(df_12,card_vc, on = 'Card Name')
full_cards

,Card Name,Title,Link,Occurrences
0,Abandon Reason,[Filter Anje cEDH Primer],[https://www.moxfield.com/decks/PFGBjM5BnESfH9...,1
1,Abeyance,[Heliod II: Sunshine Boogaloo],[https://www.moxfield.com/decks/rohOeIs-dEmA1L...,1
2,Abrade,"[Frozen Scepter Parun 2.0, Rielle Adaptative W...",[https://www.moxfield.com/decks/belqftDqDEWVEZ...,31
3,Abrupt Decay,"[Tymna/Sidar Raza, The King's Chain, Medium Mi...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...,64
4,Abundance,[Simic Ad Naus in the CZ],[https://www.moxfield.com/decks/Vva2blXPDEWTPC...,1
...,...,...,...,...
1531,Zealous Persecution,"[Tymna/Sidar Raza, The Pet Cemetery, Metapod P...",[https://www.moxfield.com/decks/1vqfLIEtWkOnZo...,3
1532,"Zirda, the Dawnwaker","[Dawnwaker Thrasios, Advantage Blue Pod (ABP)]",[https://www.moxfield.com/decks/kLygDHVCskGtvw...,2
1533,Zulaport Cutthroat,"[Turbo Poison Hulk, Demoncratic Markov, The Gr...",[https://www.moxfield.com/decks/3jJGHnq1-UO_dH...,3
1534,Zur the Enchanter,"[Consultation Zur, Shining Shimmering Splendor...",[https://www.moxfield.com/decks/1t0Cpm1xqUeyA-...,3


# EXPORT RESULTS
* JSON file with only cards and card occurrences
* JSON file with cards and decklists information
* CSV with cards and occurrences

In [36]:
card_vc.to_json('results/competitiveCards.json', orient='records')
card_vc.to_csv('results/competitive_cards.csv')
full_cards.to_json('results/competitiveCards_full.json', orient='records')